In [1]:
import os
import pickle
import google.auth
from googleapiclient.discovery import build
from google.oauth2 import service_account
from googleapiclient.errors import HttpError
import googleapiclient
import gspread
import base64
from email.mime.text import MIMEText
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import google.auth
import datetime


In [2]:
date_today = datetime.date.today().strftime("%Y-%m-%d")
blob_name = fr'GYG - {date_today}.xlsx'
blob_name_short = blob_name.split('.')[0]
file_path_logs_processed = fr'G:/.shortcut-targets-by-id/1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2/MyOTAs/Baza Excel/Logs/files_processed/{blob_name_short}'


In [3]:
#Send emial via gmail api template
def send_email(subject, message, recipient, static_discovery=False):
    # If modifying these SCOPES, delete the file token.pickle.
    SCOPES = ['https://www.googleapis.com/auth/gmail.send',
              'https://www.googleapis.com/auth/spreadsheets']

    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'client_secret.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('gmail', 'v1', credentials=creds)
    # create a MIMEText object with the message content
    message_text = MIMEText(message +"<br><br>", 'html')

    # set the recipient and subject fields
    message_text['to'] = recipient
    message_text['subject'] = subject
#     create_message = {'raw': base64.urlsafe_b64encode(message.as_bytes()).decode()}
    create_message = {'raw': base64.urlsafe_b64encode(message_text.as_bytes()).decode()}
    
    # send the message
    send_message = service.users().messages().send(userId="me", body=create_message).execute()
    print(F'The message was sent to {recipient} Message Id: {send_message["id"]}')

In [4]:
message = ''
subject = ''
#check raw
if os.path.isfile(f'{file_path_logs_processed}-raw.txt'):
    print(f'The file "{file_path_logs_processed}" exists.')
else:
    print(f'The file "{file_path_logs_processed}" does not exist.')
    message = f'{blob_name_short}-raw not exist'
    subject = f'{blob_name_short}-raw'

# check refined
if os.path.isfile(f'{file_path_logs_processed}-refined.txt'):
    print(f'The file "{file_path_logs_processed}" exists.')
else:
    print(f'The file "{file_path_logs_processed}" does not exist.')
    if len(message) == 0:
        message = f'{blob_name_short}-refined not exist'
        subject = f'{blob_name_short}-refined'
    else:
        message = message + f' | {blob_name_short}-refined not exist'
        subject = subject + f' | {blob_name_short}-refined'
    
if len(subject) != 0:
    send_email(subject, message, 'wojbal3@gmail.com')
    send_email(subject, message, 'tech@myotas.com')

    

The file "G:/.shortcut-targets-by-id/1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2/MyOTAs/Baza Excel/Logs/files_processed/GYG - 2023-07-03" does not exist.
The file "G:/.shortcut-targets-by-id/1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2/MyOTAs/Baza Excel/Logs/files_processed/GYG - 2023-07-03" does not exist.
The message was sent to wojbal3@gmail.com Message Id: 1891b6f4293fdf0a
The message was sent to tech@myotas.com Message Id: 1891b6f4450790d2
